In [213]:
import sounddevice as sd
import random
import numpy as np
import librosa

from pydub import AudioSegment
from scipy.io import wavfile
from midi2audio import FluidSynth
from midiutil import MIDIFile
from src.util.features import get_feature_vector
from scipy.spatial.distance import cosine, minkowski

from mido import MidiFile, Message, MidiTrack

In [197]:
def play(file=None, sr=44100):
    if file == None: file=WAV_FILE+".wav"
    song, fs = librosa.load(file,sr=sr)
    sd.play(song, fs)
    sd.wait()

In [4]:
def rand(low=0, high=2):
    return random.random()*(high - low + 1) + low

In [122]:
FONT = "./src/soundfonts/piano_chords.sf2"
MIDI_FILE = "./src/input/temp"
WAV_FILE = "./src/output/temp"

In [161]:
def pitch_mutation(pitch = None):
    #23, 35, 47, 95
    pitches = [47, 59, 71]
    if pitch == None:
        n = random.randrange(0, len(pitches))
        return n, pitches[n]
    else:
        m = random.choices([-1,0,1], weights=[0.2, 0.6, 0.2])[0]
        pitch += m
        if pitch == len(pitches):
            return pitch-1, pitches[ pitch - 1 ]
        if pitch < 0:
            return 0, pitches[ 0 ]
        return pitch, pitches[ pitch ]

In [145]:
def choice(arr, size=5, replace=False):
    return list(np.random.choice(arr, size=size, replace=replace))

In [192]:
def generate_notes(n: int) -> list:
    reps = list(range(0,12+1))
    
    durations = [ random.choices( 
        range(1,8),
    )[0] for _ in range(n) ]
    notes = [ 
        choice(reps, size=1)[0] for _ in range(n) 
    ]
    
    return list(zip(durations, notes))

In [181]:
def save_harmony(midi_file, out):
    with open(MIDI_FILE + out + ".midi", "wb") as output_file:
        midi_file.writeFile(output_file)
        
    song = BytesIO()
    wav_song = BytesIO()
    
    midi_file.writeFile(song)

    FluidSynth(
            sound_font=FONT
        ).midi_to_audio(MIDI_FILE + out + ".midi", WAV_FILE + out + ".wav")

In [205]:
def generate_harmony(n: int, fl_ext: str):
    notes = generate_notes(n)
    
    midi_file = MIDIFile(1)
    tempo = 120#random.randrange(30,120 + 1)
    midi_file.addTempo(0,0, tempo)
    k, pitch = pitch_mutation()
    
    cum_time = 0
    for note in notes:
        duration, note = note
        duration = 4/duration
        k, pitch = pitch_mutation(k)
        
        midi_file.addNote(0, 0, pitch+note, cum_time, duration, 100)
        cum_time+=duration
    save_harmony(midi_file, fl_ext)
    return notes

In [206]:
generate_harmony(10, "")

[(5, 6),
 (5, 12),
 (6, 5),
 (5, 6),
 (4, 10),
 (4, 8),
 (1, 9),
 (1, 11),
 (4, 5),
 (6, 10)]

In [207]:
play()

In [208]:
def generate_random_chord_progression():
    chords = [[60, 64, 67], [62, 65, 69], [64, 67, 71], [65, 69, 72]]
    return random.choices(chords, k=4)

In [222]:
m = MidiFile()

In [223]:
track = MidiTrack()
m.tracks.append(track)

In [224]:
c = generate_random_chord_progression()

In [225]:
c

[[60, 64, 67], [60, 64, 67], [60, 64, 67], [65, 69, 72]]

In [226]:
time=0
for chord in c:
    for note in chord:
        track.append(Message('note_on', note=note, velocity=64, time=time))
        time = 0
    track.append(Message('note_off', note=note, velocity=64, time=time))

In [227]:
m.save(MIDI_FILE+"midi")

In [228]:
FluidSynth(
            sound_font=FONT
        ).midi_to_audio(MIDI_FILE + ".midi", WAV_FILE + ".wav")

In [255]:
def generate_random_melody():
    # Create a new MIDI file
    mf = MIDIFile(2)
    melody_track = 0
    bassline_track = 1
    time = 0
    
    mf.addTrackName(melody_track, time, "Melody Track")
    mf.addTrackName(bassline_track, time, "Bassline Track")
    mf.addTempo(melody_track, time, 120)
    mf.addTempo(bassline_track, time, 120)

    # Generate a random chord progression
    chord_progression = generate_random_chord_progression()

    for i in range(10):
        # Add the chord to the MIDI file
        chord = chord_progression[i % len(chord_progression)]
        for n in chord:
            mf.addNote(melody_track, 0, n, i, 1, 100)

        # Generate a random melody note that fits the chord
        pitch = random.choice(chord)
        duration = 1
        mf.addNote(melody_track, 0, pitch, i, duration, 100)

        # Generate a random bassline note that fits the chord
        print(chord)
        current_chord = [note for note in chord if note % 12 in [0, 5]]
        if len(current_chord)==0:
            current_chord = [chord[0]]
        bassline_pitch = random.choice(current_chord)
        mf.addNote(bassline_track, 0, bassline_pitch, i, 1, 100)

    # Save the MIDI file
    with open(MIDI_FILE+".midi", 'wb') as outf:
        mf.writeFile(outf)
    
    FluidSynth(
            sound_font=FONT
        ).midi_to_audio(MIDI_FILE + ".midi", WAV_FILE + ".wav")

In [256]:
generate_random_melody()

[62, 65, 69]
[65, 69, 72]
[64, 67, 71]
[60, 64, 67]
[62, 65, 69]
[65, 69, 72]
[64, 67, 71]
[60, 64, 67]
[62, 65, 69]
[65, 69, 72]
